In [ ]:
using Revise

In [ ]:
using Boolean

In [ ]:
init_logic(3)
println(simplifyLogic(Meta.parse("((x1 + x2) * x3) * (x1 * 0 + x2 * 1)")))

In [ ]:
create_bool_rep("z1 + z2")

In [ ]:
z = simplifyLogic(Meta.parse("x1 * (1 ⊕ x1)"))

In [ ]:
z.args[3].args

In [ ]:
BitArray([((i-1) >> (j-1)) & 1  for i in 1:2^3, j in 1:3])

In [ ]:
simplifyLogic(Meta.parse("(z1 ⊕ z3) ⊕ (0 ⊕ z3)"))

In [ ]:
simplifyLogic(Meta.parse("((x1 + x2) * x3) * (x1 * 0 + x2 * 1)"))

In [ ]:
simplifyLogic(Meta.parse("0 + x1"))

In [ ]:
simplifyLogic(Meta.parse("x1 * (x1 + 0)")) 

In [ ]:
init_logic(3)

f1 = "x1 + x2 * x3"
f2 = "(z1 * z3) ⊕ (z2 * z3)"
zero = "((x1 + x2) * x3) ⊕ (x1 * x3 + x2 * x3)"

println("\nExamining the 'f2' formula:")
f2e = create_bool_rep(f2)
println(f2e)
print("Number of non-zero bits: ")
println(logicCount(f2e))
println("First 5 (at most 5) variable values that lead to a 'true' value are:")
println("$(f2e.var)1, $(f2e.var)2, $(f2e.var)3):")
res = nonZero(f2e, head=5)
print(res)

println("\nExamining the 'zero' formula:")
zeroe = create_bool_rep(zero)
println(zeroe)
print("Number of non-zero bits: ")
println(logicCount(zeroe))
println("First 5 (at most 5) variable values that lead to a 'true' value are:")
res = nonZero(zeroe, head=5)
typeof(res)
res
logicCount(zeroe)

In [ ]:
create_bool_rep("(z1 + z2) * z3")

In [ ]:
isEquiv(f1, f1)

In [ ]:
Meta.parse("((x1 + x2) * x3) * (x1 * 0 + x2 * 1)")

In [ ]:
simplifyLogic(Meta.parse("((x1 + x2) * x3) * (x1 * 0 + x2 * 1)"))

In [ ]:
frm="((1 + x2) * x3) * x2"
ee = simplifyLogic(Meta.parse(frm))
string(ee)

In [ ]:
    x = reverse(bitstring(UInt8(11)))

In [ ]:
simplifyLogic(Meta.parse("x1 ⊕ 0 ⊕ x1 ⊕ 1 ⊕ x1"))

### Operator Precedence: XOR lower than "*" (Both left associative)

In [ ]:
simplifyLogic(Meta.parse("1 ⊕ 1 * 0"))

In [ ]:
simplifyLogic(Meta.parse("(1 ⊕ 1) * 0"))

### Operator Precedence: XOR same as "+" (Both left associative)

#### Is "+" higher than XOR: NO

In [ ]:
simplifyLogic(Meta.parse("1 ⊕ 1 + 1"))

In [ ]:
simplifyLogic(Meta.parse("1 ⊕ (1 + 1)"))

#### Is XOR higher than "+": NO

In [ ]:
simplifyLogic(Meta.parse("1 + 1 ⊕ 1"))

In [ ]:
simplifyLogic(Meta.parse("1 + (1 ⊕ 1)"))

### Extending the Function nonZero
For future improvement of the package. 

In [ ]:
# Intension to take a formula and find inputs that return 1.
form="(x1 + (x2 * ~x5)) * (x3 + x4) +  ~x2"

# Total number of variables.
N=5
NV=3

# Init the logic system for (N-NV) variables.
init_logic(N-NV)

x = Vector{Int64}(undef, N)

# For each of the first `NV` variables
# loop through all values of `0` and `1`s.
# For each set of NV values, substitute these into `form`
# and then simply the formula.
# Remap the variables that remain to 1 to (N-NV).

d = Dict{String,String}()
for i in (NV+1):N
    d["x$i"] = "x$(i-NV)"
end

println(d)

for i in 0:(2^NV-1)
    x = reverse(bitstring(UInt8(i)))
    println("\tx = $(x[1:N])")
    frm = form
    firstBits = x[1:NV]
    for j in 1:NV
        frm = replace(frm, "x$j" => x[j])
    end
    
    eqn = simplifyLogic(Meta.parse(frm))
    if eqn == 0 || eqn == 1
        if eqn == 1
            println(x)
        end
    else
        eqn = string(eqn)
        # We need to convert the vars over to 1:(N-NV). FIXME
        for k in keys(d)
            eqn = replace(eqn, k => d[k])
        end
        println("eqn = $eqn")
        eqnb = create_bool_rep(string(eqn))
        if logicCount(eqnb) != 0
            res = nonZero(eqnb, head=5)
            println(typeof(res))
            print(string(res))
        end
    end
end

In [ ]:
ex = simplifyLogic(Boolean.fixXorParseTree(Meta.parse("x1 ⊕ x2 ⊕ x3 ⊕ x4")))

In [ ]:
ex = Meta.parse("x1 ⊕ x2 ⊕ x3 ⊕ x4")

In [ ]:
ex.args[2]

In [ ]:
z = Boolean.fixXorParseTree(ex)

In [ ]:
z.head

In [ ]:
z.args

In [ ]:
simplifyLogic(z)

In [ ]:
ex.args[2].args[1]

In [ ]:
ex = Meta.parse("x1 + x1 + x1")

In [ ]:
ex.args

In [ ]:
ex = Meta.parse("x1 ⊕ x2 ⊕ x3 ⊕ x4")

In [ ]:
z = Boolean.fixXorParseTree(ex)

In [ ]:
ex = Meta.parse("x1 ⊕ x2 ⊕ x3")

In [ ]:
zz = Boolean.fixXorParseTree(ex)

In [ ]:
zz.args

In [ ]:
z = Boolean.fixXorParseTree(ex)

In [ ]:
z.head

In [ ]:
z.args

In [ ]:
ex = Meta.parse("(x1 * x2) ⊕ (x2 ⊕ x3) ⊕ (x2 + x5) ⊕ x4")

In [ ]:
z = Boolean.fixXorParseTree(ex)

In [ ]:
z.args

In [ ]:
ex = Meta.parse("(x1 ⊕ x3) ⊕ (x1 ⊕ x3)")

In [ ]:
ex.args[2].args

In [ ]:
simplifyLogic(ex)

In [ ]:
ex = Meta.parse("x1 ⊕ (x2 + x3) ⊕ x4 ⊕ x5")

In [ ]:
zz = Boolean.fixXorParseTree(ex; verbose=true)

In [ ]:
zz.args

In [ ]:
simplifyLogic(parseLogic("x1 ⊕ (x2 + x3) ⊕ x4 ⊕ x5"))

In [ ]:
simplifyLogic(Boolean.fixXorParseTree(Meta.parse("(z1 ⊕ z3) ⊕ (0 ⊕ z3)")))

In [ ]:
simplifyLogic(Boolean.fixXorParseTree(Meta.parse("((1 + x2) * x3) * x2")))

In [ ]:
using Boolean

In [ ]:
simplifyLogic(parseLogic("(x1 * x2) + (x2 * x3) + (x1 * x2)"))

In [ ]:
z = parseLogic("(x1 * x2) + (x2 * x3) + (x1 * x2)")

In [ ]:
z.args

In [ ]:
parseLogic("x1 * x2 + x2 * x3 + x1 * x2")

In [ ]:
parseLogic("x1 * x2 + x2 * x3 + x1 * x2 + ~x10")

In [ ]:
init_logic(10)

In [ ]:
Boolean.vars

In [ ]:
sizeof(Boolean.vars)

In [ ]:
1024 * 10 / 8

In [ ]:
form = @bfunc x1 + x2 ⊕ x3 * x2 + ~x10

In [ ]:
sizeof(form)

In [ ]:
form

In [ ]:
sizeof(form.val)

In [ ]:
typeof(form)

In [ ]:
form

In [ ]:
str = "x1 + x2 ⊕ x3 * (x2 + x4)"
xx = maximum([parse(Int64, x) for x in split(str, r"[ *+~()⊕a-z]+") if x != ""])

In [ ]:
sizeof(form.val)

In [ ]:
sizeof(BitVector(Bool[0,1,1,1,0,1, 0, 0,0,1,0,1,1,1,1,0,0,0,1,0,0,1,1]))

In [ ]:
display(form)

In [ ]:
println(form.val)